## Machine Learning

In [50]:
import numpy as np
import pandas as pd

import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
preprocessed_data = pd.read_csv('Absenteeism_preprocessed.csv')

In [3]:
preprocessed_data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [4]:
preprocessed_data['Absenteeism Time in Hours'].median()

3.0

In [5]:
target = np.where(preprocessed_data['Absenteeism Time in Hours'] > 
                  preprocessed_data['Absenteeism Time in Hours'].median(), 1, 0)

In [6]:
data_with_target = preprocessed_data.drop(['Absenteeism Time in Hours'], axis=1)

In [7]:
data_with_target.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1


In [8]:
data_with_target['Excessive Absenteeism'] = target

In [9]:
data_with_target.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,0


In [10]:
data_with_target.iloc[:, :-1]

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,2,179,22,40,237.656,22,1,2,0
696,1,0,0,0,5,2,225,26,28,237.656,24,0,1,2
697,1,0,0,0,5,3,330,16,28,237.656,25,1,0,0
698,0,0,0,1,5,3,235,16,32,237.656,25,1,0,0


In [11]:
unscaled_input = data_with_target.iloc[:, :-1]

In [13]:
# absenteeism_scaler = StandardScaler()
# absenteeism_scaler.fit(unscaled_input)
# scaled_input = absenteeism_scaler.transform(unscaled_input)

In [20]:
# Custom Scalar not to use [0,1] variables 

class CustomScaler(BaseEstimator,TransformerMixin): 
    def __init__(self, columns, copy=True, with_mean=True, with_std=True):
        self.scaler = StandardScaler(copy=copy, with_mean=with_mean, with_std=with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [21]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Education']
columns_to_scale = [x for x in unscaled_input.columns.values if x not in columns_to_omit]

In [22]:
columns_to_scale

['Month Value',
 'Day of the Week',
 'Transportation Expense',
 'Distance to Work',
 'Age',
 'Daily Work Load Average',
 'Body Mass Index',
 'Children',
 'Pets']

In [23]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [24]:
absenteeism_scaler.fit(unscaled_input)
scaled_input = absenteeism_scaler.transform(unscaled_input)

C:\Users\TaimurAdil\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [35]:
scaled_input

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.030796,-0.800950,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.030796,-0.800950,-1.574681,-1.141882,2.130803,-0.806331,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.030796,-0.232900,-0.654143,1.426749,0.248310,-0.806331,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.030796,0.335149,0.854936,-1.682647,0.405184,-0.806331,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.030796,0.335149,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.568019,-0.232900,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.568019,-0.232900,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.568019,0.335149,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.568019,0.335149,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,1,-0.919030,-0.589690


In [36]:
scaled_input.shape

(700, 14)

In [37]:
x_train, x_test, y_train, y_test = train_test_split(scaled_input, target, train_size = 0.6, random_state = 20)

In [38]:
reg = LogisticRegression()
reg.fit(x_train,y_train)
reg.score(x_train,y_train)

0.7619047619047619

In [39]:
model_outputs = reg.predict(x_train)
model_outputs

array([0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,

In [40]:
feature_name = unscaled_input.columns.values

In [41]:
summary_table = pd.DataFrame (columns=['Feature name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature name,Coefficient
0,Reason_1,2.461347
1,Reason_2,0.828230
2,Reason_3,2.578594
3,Reason_4,0.569759
4,Month Value,0.037815
5,Day of the Week,-0.025146
6,Transportation Expense,0.576106
7,Distance to Work,-0.049993
8,Age,-0.141294
9,Daily Work Load Average,-0.051682


In [42]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-1.331659
1,Reason_1,2.461347
2,Reason_2,0.828230
3,Reason_3,2.578594
4,Reason_4,0.569759
5,Month Value,0.037815
6,Day of the Week,-0.025146
7,Transportation Expense,0.576106
8,Distance to Work,-0.049993
9,Age,-0.141294


#### Interpreting the coefficients

In [43]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [44]:
summary_table.sort_values('Odds_ratio', ascending=False)

,Feature name,Coefficient,Odds_ratio
3,Reason_3,2.578594,13.178592
1,Reason_1,2.461347,11.720583
2,Reason_2,0.828230,2.289263
7,Transportation Expense,0.576106,1.779096
4,Reason_4,0.569759,1.767841
13,Children,0.319644,1.376637
11,Body Mass Index,0.250808,1.285064
5,Month Value,0.037815,1.038539
6,Day of the Week,-0.025146,0.975167
8,Distance to Work,-0.049993,0.951236


In [45]:
reg.score(x_test,y_test)

0.7571428571428571

In [47]:
predicted_proba = reg.predict_proba(x_test)
#predicted_proba

In [48]:
predicted_proba[:,1]


array([0.257732  , 0.38492868, 0.53578299, 0.24866403, 0.90856328,
       0.69909124, 0.68557443, 0.85394273, 0.24448883, 0.25839542,
       0.43350741, 0.80385645, 0.9011497 , 0.27094595, 0.69147691,
       0.45040707, 0.43418102, 0.37588341, 0.66258338, 0.91751881,
       0.25894418, 0.25134233, 0.56507157, 0.56917608, 0.71939112,
       0.22892795, 0.52704778, 0.12177674, 0.71735167, 0.25403973,
       0.3983591 , 0.67175573, 0.66622819, 0.43692511, 0.25134233,
       0.56411942, 0.26012599, 0.73469305, 0.4585584 , 0.5860095 ,
       0.24707393, 0.43697249, 0.234106  , 0.43679214, 0.77501459,
       0.60593319, 0.74028694, 0.24329232, 0.29309703, 0.22331944,
       0.487793  , 0.31406961, 0.66200685, 0.27698927, 0.82465366,
       0.40715575, 0.86302583, 0.29292407, 0.3763565 , 0.38455241,
       0.68392876, 0.67943458, 0.2976902 , 0.73469124, 0.24343119,
       0.26227745, 0.11116478, 0.25014915, 0.74596986, 0.28124489,
       0.23816131, 0.3619239 , 0.89024625, 0.44579631, 0.58022

In [51]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [52]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)